### Mutable sequence types

In [10]:
a = [1,2,3]
a.extend('bcd')
a

[1, 2, 3, 'b', 'c', 'd']

In [11]:
a.reverse()
a

['d', 'c', 'b', 3, 2, 1]

In [8]:
l = [1,2,3]
l.extend({5,6,7,'a', 'c', 'b'})
l  # order is not preserved!

[1, 2, 3, 5, 6, 'a', 7, 'b', 'c']

### Lists vs tuples

**Constant folding** is the process of recognizing and evaluating constant expressions at COMPILE TIME rather than computing them at RUNTIME

In [12]:
from dis import dis
(1,2,3)

(1, 2, 3)

In [13]:
[1,2,3]

[1, 2, 3]

In [15]:
dis(compile('(1,2,3, "a")', 'string', 'eval'))  # immutable

  1           0 LOAD_CONST               0 ((1, 2, 3, 'a'))
              2 RETURN_VALUE


In [16]:
dis(compile('[1,2,3, "a"]', 'string', 'eval'))  # mutable

  1           0 LOAD_CONST               0 (1)
              2 LOAD_CONST               1 (2)
              4 LOAD_CONST               2 (3)
              6 LOAD_CONST               3 ('a')
              8 BUILD_LIST               4
             10 RETURN_VALUE


In [17]:
from timeit import timeit

In [18]:
timeit("(1,2,3,4,5,6,7,8,9)", number=10_000_000)

0.14600869999992483

In [19]:
timeit("[1,2,3,4,5,6,7,8,9]", number=10_000_000)

1.299367200000006

In [ ]:
# If you don't need mutability, you're better off with tuples!

In [20]:
timeit('tuple((1,2,3,4,5))', number=5_000_000)  # doesn't construct a new tuple - same memory id

0.3479366999999911

In [21]:
timeit('list((1,2,3,4,5))', number=5_000_000)  # creates a new list - new memory id

1.07511280000017

In [22]:
t = tuple(range(100_000))
l = list(t)

In [26]:
timeit('t[-1]', globals=globals(), number=5_000_000)

0.1984577000000627

In [27]:
timeit('l[-1]', globals=globals(), number=5_000_000)

0.22276640000018233

### Copying Sequences

In [1]:
l1 = [1,2,3]
l1_copy = l1.copy()

print(id(l1), id(l1_copy))

2438393668032 2438380058688


In [2]:
l1_copy = list(l1)
print(id(l1), id(l1_copy))

2438393668032 2438380671104


In [5]:
t1 = (1,2,3)
t2 = tuple(t1)
print(id(t1), id(t2))  # same memory address - doens't make sense to make a copy of an immutable sequence

2438394667648 2438394667648


In [6]:
l1 = [1,2,3]
l2 = l1[:]
print(id(l1), id(l2))  # slicing mutable sequences returns different objects

2438393696000 2438393734144


In [7]:
s1 = 'python'
s2 = str(s1)
print(id(s1), id(s2))

2438334260016 2438334260016


In [8]:
import copy
l1 = [1,2,3]
l2 = copy.copy(l1)
print(id(l1), id(l2))

2438393693568 2438394047488


In [9]:
v1 = [0,0]
v2 = [0,0]
line1 = [v1, v2]
line2 = line1.copy()
print(id(line1), id(line2))  # shallow copy stores sequences of mutable objects under diff memory addresses

2438393692224 2438380056896


In [10]:
print(id(line1[0]), id(line2[0]))  # elements in each sub-list have the same memory address

2438374962688 2438374962688


In [13]:
line1[0][0] = 100
print(line1)
print(line2)  # oops!

[[100, 0], [0, 0]]
[[100, 0], [0, 0]]


In [16]:
v1 = [1,1]
v2 = [2,2]
v3 = [3,3]
v4 = [4,4]
line1 = [v1, v2]
line2 = [v3, v4]
plane1 = [line1, line2]

plane2 = copy.deepcopy(plane1)

In [17]:
print(id(plane1), id(plane2))


2438380669184 2438380672128


In [18]:
id(plane1[0]), id(plane2[0])

(2438394771456, 2438393669376)

In [19]:
id(plane1[0][0]), id(plane2[0][0])

(2438394771008, 2438380056896)

In [21]:
plane1[0][0][0] = 100
print(plane1)
print(plane2)  # edits to plane1 don't affect plane2

[[[100, 1], [2, 2]], [[3, 3], [4, 4]]]
[[[1, 1], [2, 2]], [[3, 3], [4, 4]]]


In [22]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point({self.x}, {self.y})'
    
class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __repr__(self):
        return f'Line({self.p1.__repr__()}, {self.p2.__repr__()})'    

In [25]:
p1 = Point(0,0)
p2 = Point(10,10)
line1 = Line(p1, p2)
line2 = copy.deepcopy(line1)
line3 = copy.copy(line1)

In [27]:
print('Different id with deep copy')
print(line1.p1, id(line1.p1))
print(line2.p1, id(line2.p1))

print('\nSame id with shallow copy')
print(line1.p1, id(line1.p1))
print(line3.p1, id(line3.p1))

Different id with deep copy
Point(0, 0) 2438393750576
Point(0, 0) 2438393749904

Same id with shallow copy
Point(0, 0) 2438393750576
Point(0, 0) 2438393750576


### Custom Sequences (Immutable)
- REQUIRES a __getitem__() method
- best practice to include a __len__() method

In [28]:
l = [1,2,3,4,5]
len(l), l.__len__()

(5, 5)

In [30]:
l[2], l.__getitem__(2)

(3, 3)

In [32]:
l[::-1], l.__getitem__(slice(None, None, -1))

([5, 4, 3, 2, 1], [5, 4, 3, 2, 1])

In [36]:
for i in l:
    print(i ** 2)

1
4
9
16
25


In [35]:
# Repeat the above without using a for loop
i = 0
while True:
    try:
        print(l.__getitem__(i) ** 2)
    except IndexError:
        break
    i += 1

1
4
9
16
25


In [46]:
# Sequences must have a __getitem__ method and should have a __len__ method
class Silly:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        print('Called __len__')
        return self.n
    
    def __getitem__(self, value):
        # print(f'You requested item at index {value}')
        if value < 0 or value >= self.n:
            raise IndexError
        else:
            return 'this is a silly element'

In [47]:
silly = Silly(10)
len(silly)

Called __len__


10

In [48]:
silly.__getitem__(100)

IndexError: 

In [49]:
for item in silly: print(item)

this is a silly element
this is a silly element
this is a silly element
this is a silly element
this is a silly element
this is a silly element
this is a silly element
this is a silly element
this is a silly element
this is a silly element


In [51]:
[item * 2 for item in silly]

['this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element',
 'this is a silly elementthis is a silly element']

In [54]:
silly[0:3:2]

TypeError: '<' not supported between instances of 'slice' and 'int'

In [55]:
from functools import lru_cache

@lru_cache(2 ** 10)
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [72]:
class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0:
                s = self.n + s
            if s < 0 or s >= self.n:
                raise IndexError
            return self._fib(s)
        else:  # assume it's a slice
            pass
    @staticmethod
    @lru_cache(2 ** 10)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)

In [69]:
f = Fib(8)
f[0], f[7]

(1, 21)

In [70]:
f[8]

IndexError: 

In [66]:
list(f)
f[-1]  # we aren't handling negative indicies

IndexError: 

In [67]:
[item ** 2 for item in f]

[1, 1, 4, 9, 25, 64, 169, 441]

In [73]:
fib = Fib(10)
list(fib)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [74]:
fib[0], fib[9], fib[-1]

(1, 55, 55)

In [76]:
fib[0:5]  # time to implement slices

In [77]:
class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0:
                s = self.n + s
            if s < 0 or s >= self.n:
                raise IndexError
            return self._fib(s)
        else:  # assume it's a slice
            start, stop, step = s.indices(self.n)
            rng = range(start, stop, step)
            # rng = range(*s.indices(self.n))
            return [self._fib(i) for i in rng]
    @staticmethod
    @lru_cache(2 ** 10)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)

In [79]:
f = Fib(12)
list(f)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]

In [80]:
f[0:4]

[1, 1, 2, 3]

In [ ]:
f[-1:-4:-1]

[144, 89, 55]

### Custom Sequences (MUTABLE)

In [1]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'MyClass(name={self.name})'
    
    def __add__(self, other):
        print(f'You called + on {self} and {other}')
        return 'hello from __add__'
    
    def __iadd__(self, other):
        print(f'You called += on {self} and {other}')
        return 'hello from __iadd__'

In [2]:
c1 = MyClass('instance1')
c2 = MyClass('instance2')

In [3]:
c1 + c2

You called + on MyClass(name=instance1) and MyClass(name=instance2)


'hello from __add__'

In [4]:
id(c1)

2783794119728

In [5]:
c1 += c2

You called += on MyClass(name=instance1) and MyClass(name=instance2)


In [6]:
id(c1)

2783812432880

In [7]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'MyClass(name={self.name})'
    
    def __add__(self, other):
        return MyClass(self.name + other.name)
    
    def __iadd__(self, other):
        if isinstance(other, MyClass):
            self.name += other.name
        else:
            self.name += other
        return self

In [8]:
c1 = MyClass('Eric')
c2 = MyClass('Idle')

In [9]:
id(c1), id(c2)

(2783795083680, 2783795083344)

In [10]:
result = c1 + c2
print(id(result), result)  # id changes

2783795083872 MyClass(name=EricIdle)


In [11]:
c1 += c2
print(id(c1), c1)  # id does not change

2783795083680 MyClass(name=EricIdle)


In [12]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'MyClass(name={self.name})'
    
    def __add__(self, other):
        return MyClass(self.name + other.name)
    
    def __iadd__(self, other):
        if isinstance(other, MyClass):
            self.name += other.name
        else:
            self.name += other
        return self
    
    def __mul__(self, n):
        return MyClass(self.name * n)
    
    def __imul__(self, n):
        self.name *= n
        return self

In [13]:
c1 = MyClass('Eric')
id(c1)

2783795085216

In [14]:
result = c1 * 3
id(result), result  # new memory address

(2783795083584, MyClass(name=EricEricEric))

In [15]:
c1 *= 3
id(c1), c1  # same memory address, name has changed

(2783795085216, MyClass(name=EricEricEric))

In [17]:
c1 = MyClass('Eric')
3 * c1  # 3.__mul__(c1) NotImplemented and c1.__rmul__(3) AttributeError

TypeError: unsupported operand type(s) for *: 'int' and 'MyClass'

In [18]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'MyClass(name={self.name})'
    
    def __add__(self, other):
        return MyClass(self.name + other.name)
    
    def __iadd__(self, other):
        if isinstance(other, MyClass):
            self.name += other.name
        else:
            self.name += other
        return self
    
    def __mul__(self, n):
        return MyClass(self.name * n)

    def __rmul__(self, n):
        return self.__mul__(n)
    
    def __imul__(self, n):
        self.name *= n
        return self

In [19]:
c1 = MyClass('Eric')
c1 * 3

MyClass(name=EricEricEric)

In [20]:
3 * c1

MyClass(name=EricEricEric)

In [21]:
class MyClass:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'MyClass(name={self.name})'
    
    def __add__(self, other):
        return MyClass(self.name + other.name)
    
    def __iadd__(self, other):
        if isinstance(other, MyClass):
            self.name += other.name
        else:
            self.name += other
        return self
    
    def __mul__(self, n):
        return MyClass(self.name * n)

    def __rmul__(self, n):
        return self.__mul__(n)
    
    def __imul__(self, n):
        self.name *= n
        return self
    
    def __contains__(self, value):
        return value in self.name

In [22]:
c1 = MyClass('Eric Idle')
'Eric' in c1

True

In [23]:
'I' in c1

True

### Sorting

In [3]:
sorted(['ghi', 'def'])

['def', 'ghi']

In [4]:
from timeit import timeit
import random

In [9]:
random.seed(0)
n = 10_000_000
l = [random.randint(0,100) for n in range(n)]
l[:10]

[49, 97, 53, 5, 33, 65, 62, 51, 100, 38]

In [10]:
timeit('sorted(l)', globals=globals(), number=1)

1.8176180999998905

In [11]:
timeit('l.sort()', globals=globals(), number=1)

1.8181346000001213

In [13]:
class MyClass:
    def __init__(self, name, val):
        self.name = name
        self.val = val

    def __repr__(self):
        return f'MyClass({self.name}, {self.val})'

    def __lt__(self, other):
        return self.val < other.val

In [14]:
c1 = MyClass('c1', 20)
c2 = MyClass('c2', 10)
c3 = MyClass('c3', 20)
c4 = MyClass('c4', 10)
sorted([c1, c2, c3, c4])

[MyClass(c2, 10), MyClass(c4, 10), MyClass(c1, 20), MyClass(c3, 20)]

In [15]:
c1 < c2

False

In [16]:
l = [c4, c2, c3, c1]
sorted(l)

[MyClass(c4, 10), MyClass(c2, 10), MyClass(c3, 20), MyClass(c1, 20)]

In [18]:
sorted(l, key=lambda x: x.name, reverse=True)

[MyClass(c4, 10), MyClass(c3, 20), MyClass(c2, 10), MyClass(c1, 20)]

### List Comprehensions

In [19]:
compiled_code = compile('[i**2 for i in (1,2,3)]', filename='string', mode='eval')

In [20]:
compiled_code

<code object <module> at 0x00000157DE075190, file "string", line 1>

In [21]:
import dis
dis.dis(compiled_code)

  1           0 LOAD_CONST               0 (<code object <listcomp> at 0x00000157DE075030, file "string", line 1>)
              2 LOAD_CONST               1 ('<listcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_CONST               2 ((1, 2, 3))
              8 GET_ITER
             10 CALL_FUNCTION            1
             12 RETURN_VALUE

Disassembly of <code object <listcomp> at 0x00000157DE075030, file "string", line 1>:
  1           0 BUILD_LIST               0
              2 LOAD_FAST                0 (.0)
        >>    4 FOR_ITER                12 (to 18)
              6 STORE_FAST               1 (i)
              8 LOAD_FAST                1 (i)
             10 LOAD_CONST               0 (2)
             12 BINARY_POWER
             14 LIST_APPEND              2
             16 JUMP_ABSOLUTE            4
        >>   18 RETURN_VALUE


In [23]:
l1 = ['a', 'b', 'c']
l2 = ['x', 'y', 'z']
# we want 'ax', 'ay', 'az', ..., 'cy', 'cz'

In [25]:
[left + right for left in l1 for right in l2]

['ax', 'ay', 'az', 'bx', 'by', 'bz', 'cx', 'cy', 'cz']

In [26]:
l1 = [1,2,3,4,5,6,7,8,9]
l2 = ['a', 'b', 'c', 'd']
list(zip(l1, l2))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

In [28]:
[(val1,val2) for i1,val1 in enumerate(l1) for i2,val2 in enumerate(l2) if i1 == i2]

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]